In [ ]:
class ValueIteration:
    def __init__(self, times, time):
        self.n = len(times)
        self.times = times
        self.value = [0 for j in range(self.n)]
        self.policy = [None for j in range(self.n)]
        self.time = time
        self.bunnies = []
    
    def argmax(self, a):
        amax = None
        m = 0
        for i in range(len(a)):
            m = max([m, a[i]])
            if m == a[i]:
                amax = 1
        return amax
    
    def argmin(self, a):
        amin = None
        m = float("inf")
        for i in range(len(a)):
            m = min([m, a[i]])
            if m == a[i]:
                amin = 1
        return amax        
    
    def get_value(self, state_idx, action):
        new_bunny = ((state_idx - 1) in self.bunnies) * 10
        time_spent = action
        return self.time - time_spent + new_bunny
    
    def bellman_optimal(self, current_value, state_idx):
        # the value of this state_action pair
        # is the value of this state plus the best value 
        # of the actions we can take
        # plus the remaining time
        for action in self.times[state_idx]:
            v = self.get_value(state_idx, action)
        self.value[state_idx] = max(v)
        return max(r)
        
    def step(self, s):
        for i in range(len(s)):
            self.value[s][a] = min(self.times[s])
            self.policy[s] = self.argmin(self.times[s])
    
    def solve(self, n_iter):
        for i in range(n_iter):
            for s in range(len(self.times)):
                self.step(s)
        return            

In [ ]:
from itertools import chain, combinations, permutations

def trajectories(n_bunnies):
    """
    Returns a list of all possible trajectories, given the number of bunnies.
    It includes partial trajectories, e.g, [0, 1] in case of 3 bunnies.
    Args:
        n_bunnies (int): the total number of bunnies in the game
    Returns:
        (iterator): Returns all the possible trajectories in episode
    """
    states = range(n_bunnies)
    sets = list(chain.from_iterable(combinations(states, r) for r in range(len(states) + 1)))
    ts = set()
    for s in sets:
        ts.update(permutations(s))
    return ts
   

def floyd_warshall(times):
    """
    Finds the all-pairs shortest paths in the graph using the Floyd-Warshall algorithm.
    For a matrix u x v it stores the shortest path from u to v in m[u][v].
    Achieves O(V^3).
    Args:
        times (List[List[int]]): The adjacency matrix of the weighted graph
    Returns:
        A matrix of the same size of the adjacency matrix containing the shortest paths from
        a state to another. I.e. delta[u][v] contains the shortest distance from u to v.
    """
    d = len(times)
    delta = [[times[u][v] for v in range(len(times[u]))] for u in range(len(times))]
    for k in range(d):
        for u in range(d):
            for v in range(d):
                if delta[u][v] > delta[u][k] + delta[k][v]:  # is it shorter?
                    delta[u][v] = delta[u][k] + delta[k][v]
                    
    for u in range(d):
        if delta[u][u] < 0:  # is there a negative cycle to itself?
            return None            
    return delta


def run(trajectory, deltas):
    """
    Makes a run, starting from the door and getting to the exit, and passing
    through the bunnies asked to rescue in the trajectory
    Args:
        trajectory (List[int]): The list of bunnies id to rescue
        deltas (List[List[int]]): The square matrix of shortest paths to another state
    Returns:
        (int): The time spent to walk through the hallway, starting from the door, saving the 
        requested bunnies and getting to the door
    """
    elapsed = 0
    parent = 0  # start at the door
    # get the bunnies 
    for bunny_id in trajectory:
        state_id = bunny_id + 1  # skip the door
        elapsed += deltas[parent][state_id]
        parent = state_id
    # got to the exit
    elapsed += deltas[parent][len(deltas) - 1]
    return elapsed


def optimise(deltas, time_limit):
    """
    Given the matrices of deltas fo a graph and the max number of moves to make, it returns the
    highest number of bunnies to be able to catch.
    It iterates over all the possible trajectories and returns the one that provides most bunnies.
    Args:
        deltas (List[List[int]]) a square matrix containing the shortest paths for all pair-wise
        combinations of nodes in the graph
        time_limit (int): the max distance allowed to go, while saving the bunnies
    Returns:
        (List[int]): A list containing the ids of the bunnies we have been able to save
    """
    n = len(deltas) - 2
    bunnies = [x for x in range(len(deltas) - 2)]
    trajs = trajectories(len(bunnies))

    best_rescue = []
    for traj in sorted(trajs):
        print(str(traj))
        # get the total time spent for the rescue run
        elapsed = run(traj, deltas)
        print("elapsed", elapsed)
        # how many bunnies am I saving?
        n_saved = len(traj)
        # did I exceed time limit?
        print("out of time", time_limit, ">", elapsed, elapsed > time_limit)
        if elapsed > time_limit:
            continue
        # ok, we made it in time, is this rescue better than the current best?
        print("better rescue", n_saved, ">", len(best_rescue), n_saved > len(best_rescue))
        if n_saved > len(best_rescue):
            best_rescue = traj
            # we got a better choice, have I already saved all bunnies?
            if n_saved == len(bunnies):
                return traj
        print("---")
    print(best_rescue)
    return sorted(best_rescue)

        
def solution(times, time_limit):
    if len(times) <= 2:
        return []
    deltas = floyd_warshall(times)
    if deltas is None:
        return list(range(len(times) - 2))
    bunnies = optimise(deltas, time_limit)
    return sorted(bunnies)


def test(m, t, a):
    r = solution(m, t)
    assert r == a, "{} different from {}".format(r, a)

In [ ]:
test([[0,  1,  5,  5,  2],
          [10, 0,  2,  6,  10],
          [10, 10, 0,  1,  5],
          [10, 10, 10, 0,  1],
          [10, 10, 10, 10, 0]], 5, [0, 1, 2])

In [ ]:
    test([[0, 1, 3, 4, 2],
          [10, 0, 2, 3, 4],
          [10, 10, 0, 1, 2],
          [10, 10, 10, 0, 1],
          [10, 10, 10, 10, 0]], 4, [])            

In [ ]:
    test([[0, 2, 2, 2, -1],
          [9, 0, 2, 2, -1],
          [9, 3, 0, 2, -1],
          [9, 3, 2, 0, -1],
          [9, 3, 2, 2, 0]], 1, [1, 2])

In [ ]:
test([[0,  1, 10, 10, 10],
          [10, 0,  1,  1,  2],
          [10, 1,  0, 10, 10],
          [10, 1,  10, 0, 10],
          [10, 10, 10, 10, 0]], 7, [0, 1, 2])

In [ ]:
test([[0, 1, 1, 1, 1],
          [1, 0, 1, 1, 1],
          [1, 1, 0, 1, 1],
          [1, 1, 1, 0, 1],
          [1, 1, 1, 1, 0]], 3, [0, 1])

In [ ]:
test([[0, 5, 11, 11, 1],
          [10, 0, 1, 5, 1],
          [10, 1, 0, 4, 0],
          [10, 1, 5, 0, 1],
          [10, 10, 10, 10, 0]], 10, [0, 1])

In [ ]:
a = test([[0, 20, 20, 20, -1],
      [90, 0, 20, 20, 0],
      [90, 30, 0, 20, 0],
      [90, 30, 20, 0, 0],
      [-1, 30, 20, 20, 0]], 0, [0, 1, 2])
print(a)

In [ ]:
test([[0, 10, 10, 10, 1],
          [0, 0, 10, 10, 10],
          [0, 10, 0, 10, 10],
          [0, 10, 10, 0, 10],
          [1, 1, 1, 1, 0]], 5, [0, 1])

In [ ]:
test([[2, 2],
          [2, 2]], 5, [])

In [ ]:
test([[0, 10, 10, 1, 10],
     [10, 0, 10, 10, 1],
     [10, 1, 0, 10, 10],
     [10, 10, 1, 0, 10],
     [1, 10, 10, 10, 0]], 6, [0, 1, 2])

In [ ]:
test([[1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1]], 1, [])

In [ ]:
test([[0, 0, 1, 1, 1],
          [0, 0, 0, 1, 1],
          [0, 0, 0, 0, 1],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]], 0, [0, 1, 2])

In [ ]:
test([[1, 1, 1, 1, 1],
      [-1, 1, 1, 1, 1],
      [-1, 1, 1, 1, 1],
      [-1, 1, 1, 1, 1],
      [-1, 1, 1, 1, 1]], 1, [0, 1, 2])

In [ ]:
test([[0, 1, 5, 5, 5, 5],
      [5, 0, 1, 5, 5, 5],
      [5, 5, 0, 5, 5, -1],
      [5, 5, 1, 0, 5, 5],
      [5, 5, 1, 5, 0, 5],
      [5, 5, 1, 1, 1, 0]]
      , 3, [0, 1, 2, 3])

In [ ]:
test([[0, 1, 5, 5, 5, 5, 5],
      [5, 0, 1, 5, 5, 5, 5],
      [5, 5, 0, 5, 5, 0, -1],
      [5, 5, 1, 0, 5, 5, 5],
      [5, 5, 1, 5, 0, 5, 5],
      [5, 5, 0, 5, 5, 0, 0],
      [5, 5, 1, 1, 1, 0, 0]]
      , 3, [0, 1, 2, 3, 4])

In [ ]:
test([[0,-1, 0, 9, 9, 9, 9, 9],  # Start
      [9, 0, 1, 9, 9, 9, 9, 9],  # 0
      [0, 9, 0, 0, 9, 9, 1, 1],  # 1
      [9, 9, 9, 0, 1, 9, 9, 9],  # 2
      [9, 9, 9, 9, 0,-1, 9, 9],  # 3 
      [9, 9, 0, 9, 9, 0, 9, 9],  # 4
      [9, 9,-1, 9, 9, 9, 0, 9],  # 5
      [9, 9, 9, 9, 9, 9, 9, 0]], # bulkhead
      1, [0, 1, 2, 3, 4, 5])

In [ ]:
test([[0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0]], 0, [0, 1, 2])

<br>
<br>
<br>
<br>
<hr>
<br>
<br>
<br>
<br>

In [ ]:
from copy import deepcopy
from itertools import permutations

# I am not happy with this implementation, due to a realization I have recently had:
#   My Bellman-Ford implementation is completely unnecessary. The Floyd algorithm can detect negative cycles, and
#   negative cycles are all I used Bellman-Ford for. Thus, with some refactoring, I could easily make this more
#   efficient.


def powerset(list):
    """
    :param list: The list to generate subsets of.
    :return: A generator that produces all subsets of this set.
    """
    x = len(list)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, list) if i & mask]


def getneighbourindex(neighbour, graphsize):
    if neighbour == "Bulkhead":
        return graphsize - 1
    elif neighbour == "Start":
        return 0
    else:
        return int(neighbour)+1


def matrix2graph(matrix):
    """
    This helper function turns our matrix into a graph that's a little easier to work with using Bellman-Ford.
    :param matrix: The original matrix.
    :return: matrix in dictionary format.
    """
    keys = ["Start"]
    for num in range(1, len(matrix)-1):
        keys.append(num-1)
    keys.append("Bulkhead")
    graph = dict(zip(keys, matrix))
    return graph


# Step 1: Initialize graph
def initialize(graph, source):
    """
    Step 1 of the Bellman-Ford algorithm.
    """
    distance = {}
    predecessor = {}
    for node in graph:
        # We start off assuming all nodes are too far away!
        distance[node] = 1000
        predecessor[node] = None
    distance[source] = 0 # For the source we know how to reach
    return distance, predecessor


def relax(node, neighbour, graph, distance, predecessor):
    nidx = getneighbourindex(neighbour, len(graph))
    if distance[node] + graph[node][nidx] < distance[neighbour]:
        distance[neighbour] = distance[node] + graph[node][nidx]
        predecessor[neighbour] = node


def bellman_ford(matrix, graph, time_limit, source):
    dist, pred = initialize(graph, source)
    for num in range(len(graph)-1):
        for node in graph:
            temp = dict(graph)
            del temp[node]
            for neighbour in temp:
                # Step 2: Relax edges repeatedly
                relax(node, neighbour, graph, dist, pred)

    # Step 3: Check for negative-weight cycles
    for node in graph:
        for neighbour in graph:
            nidx = getneighbourindex(neighbour, len(graph))
            if dist[node] + graph[node][nidx] < dist[neighbour]:
                # We found a negative cycle. Since the door is open forever, free all the bunnies!~
                return [num for num in range(0, len(graph)-2)]

    # If we reach this point, it's time to employ floyd and also enumerate path lengths.
    spaths = floyd(matrix)
    # Uncomment the below code to see the floyd algorithm printed in a nice format.
    # for i in range(len(spaths)):
    #     print(spaths[i])
    return find_most_bunnies(matrix, spaths, time_limit)


def floyd(matrix):
    """
    Floyd's algorithm, straight from a textbook. Floyd's algorithm transforms a weight matrix
    into a matrix of shortest paths, such that the shortest path from node M to node N is
    equal to matrix[m][n]
    :return: An array of shortest-path distance calculations.
    """
    n = len(matrix)
    spaths = deepcopy(matrix)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if spaths[i][k] + spaths[k][j] < spaths[i][j]:
                    spaths[i][j] = spaths[i][k] + spaths[k][j]
    return spaths


def find_most_bunnies(matrix, spaths, time_limit):
    """
    And now, yet more inefficient bruteforcing to solve our NP-Hard problem. Enumerate all possible subsets,
    and then evaluate all their permutations to find the most efficient path.
    :param matrix: The original weighted matrix we will analyze with our algorithm.
    :param spaths: An array of shortest paths generated by the floyd's algorithm.
    :param time_limit: The time limit each subset is tested against.
    :return: The lexicographically least subset of bunnies that can escape.
    """
    n = len(matrix)-2
    bunnyids = []
    for num in range(n):
        bunnyids.append(num)
    pset = powerset(bunnyids)
    pset = sorted(pset)
    # Now that I've got all our possible subsets, I can calculate the distance of each path and determine which is
    # optimal.
    optimal_bunnies = []
    for sub in pset:
        for permutation in permutations(sub):
            # print(permutation)
            subsum = 0
            prev = 0
            next = len(matrix)-1
            for bunnyid in permutation:
                next = bunnyid+1
                subsum += spaths[prev][next]
                prev = next
            subsum += spaths[prev][len(matrix)-1]
            if subsum <= time_limit and len(sub) > len(optimal_bunnies):
                optimal_bunnies = sub
                if len(optimal_bunnies) == n:
                    break
            else:
                # Either rescue takes too long, or lexicographically lesser solution of same length exists.
                pass
    return optimal_bunnies


def answer(times, time_limit):
    # I was told when I got my degree I would never be asked to solve the Traveling Salesman Problem.
    # I was misinformed, but this was fun!
    if len(times) <= 2:
        return []
    graph = matrix2graph(times)
    return bellman_ford(times, graph, time_limit, "Start")